# Question 1: BigQuery and SQL

## Python Solutions


To test my sql queries I wrote python scripts to interface directly with the Big Query API.  This permitted me to run a test driven development environment to automate most of the work.  

### Import Statements

In [1]:
from google.cloud import bigquery
from pprint import pprint
import os

/Users/jnapolitano/venvs/finance/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="/Users/jnapolitano/.creds/creds.json"

### Utility Big Query Function

In [3]:
def query_big_query(query_string):
    client = bigquery.Client()
    query_job = client.query(query_string)
    results = query_job.result()  # Waits for job to complete.
    return results

## Question 1
1) Use the publicly available BigQuery dataset named `nyc-tlc.green.trips_2015`, provide SQL queries to answer the following questions:
* What is the total amount and passenger counts for the months of February, March and April?
* What has been the average hourly passenger count throughout the year?  
* What has been the change/delta in total amount billed over days? What we would like see is how much (positive or negative)
* What hour of the day has seen the longest rides in April?

## Question 1.1 
* What is the total amount and passenger counts for the months of February, March and April?

### Discussion of the Problem

I initally approached this problem somewhat naively.  I did not consider the possibliity of taxi rides overlapping per hour.  

For example the code below simply considered the pickup_datetime in the analysis.

In [4]:
def naive_query_passengers_by_month():
    query_string = """
        SELECT date_trunc(dropoff_datetime,MONTH) as Month,
        sum(passenger_count) as Sum_PASS,
        sum(total_amount) as TOTAL_AMOUNT_SUM
        FROM `nyc-tlc.green.trips_2015` 
        where dropoff_datetime BETWEEN '2015-02-01' AND '2015-04-30'
        GROUP BY Month;
        """

    result = query_big_query(query_string = query_string)

    return result.to_dataframe()
naive_df = naive_query_passengers_by_month()    

In [5]:
naive_df 

Month  Sum_PASS  TOTAL_AMOUNT_SUM
0 2015-03-01 00:00:00+00:00   2373043      2.512727e+07
1 2015-04-01 00:00:00+00:00   2220040      2.384763e+07
2 2015-02-01 00:00:00+00:00   2170450      2.281705e+07

I then attempted to create intervals to unpiviot the table with the following code.  

In [6]:
def total_passenger_total_ammount():

    query_string = """
        #standardSQL
        select timestamp_trunc(int, month) month, 
        count(pickup_datetime) rides,
        sum(passenger_count) as passenger_count,
        avg(passenger_count) as avg_passenger_count,
        sum(total_amount) as total_amount,
        avg(total_amount) as avg_total_amount
        from `nyc-tlc.green.trips_2015`, 
        unnest(generate_timestamp_array(
        pickup_datetime, 
        dropoff_datetime, 
        interval 1 hour)) int
        where pickup_datetime BETWEEN '2015-02-01' AND '2015-04-30'
        group by month
        order by month
         """
    result = query_big_query(query_string)
    return result.to_dataframe()

df_first_try = total_passenger_total_ammount()

In [7]:
df_first_try

month    rides  passenger_count  avg_passenger_count  \
0  2015-02-01 00:00:00+00:00  1612934          2226402             1.380343   
1  2015-03-01 00:00:00+00:00  1777205          2448797             1.377892   
2  2015-04-01 00:00:00+00:00  1667508          2298610             1.378470   
3  2015-05-01 00:00:00+00:00     1488             1488             1.000000   
4  2015-06-01 00:00:00+00:00     1440             1440             1.000000   
..                       ...      ...              ...                  ...   
70 2020-12-01 00:00:00+00:00     1488             1488             1.000000   
71 2021-01-01 00:00:00+00:00     1488             1488             1.000000   
72 2021-02-01 00:00:00+00:00     1344             1344             1.000000   
73 2021-03-01 00:00:00+00:00     1488             1488             1.000000   
74 2021-04-01 00:00:00+00:00      138              138             1.000000   

    total_amount  avg_total_amount  
0   2.378246e+07         14.744847  
1   2.647535e+07         14.897184  
2   2.526557e+07         15.151694  
3   5.803200e+03          3.900000  
4   5.616000e+03          3.900000  
..           ...               ...  
70  5.803200e+03          3.900000  
71  5.803200e+03          3.900000  
72  5.241600e+03          3.900000  
73  5.803200e+03          3.900000  
74  5.382000e+02          3.900000  

[75 rows x 6 columns]

### The Problem with this Approach.

Firstly the data extends to 2021 for some reason.  There are also negative values within the return set.  I spent a full day tinkering the query to clean the data.  

### Cleaning the data

In [8]:
def clean_up_data():

  query_string = """
      SELECT 
      t.*,
      FROM
      (
      SELECT *,
      TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) as time_duration_in_secs,
      TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,MINUTE) as time_duration_in_mins,
      ROUND(trip_distance/TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND),2)*3600 as driving_speed_miles_per_hour,
      (CASE WHEN total_amount=0 THEN 0
      ELSE ROUND(tip_amount*100/total_amount,2) END) as tip_rate,
      EXTRACT(YEAR from pickup_datetime) as pickup_year,
      EXTRACT(MONTH from pickup_datetime) as pickup_month,
      CONCAT(CAST(EXTRACT(YEAR from pickup_datetime) as STRING),"-",CAST(EXTRACT(MONTH from pickup_datetime) AS STRING)) as pickup_yearmonth,
      EXTRACT(DATE from pickup_datetime) as pickup_date,
      FORMAT_DATE('%A',DATE(pickup_datetime)) as pickup_weekday_name,
      EXTRACT(HOUR from pickup_datetime) as pickup_hour,
      EXTRACT(YEAR from dropoff_datetime) as dropoff_year,
      EXTRACT(MONTH from dropoff_datetime) as dropoff_month,
      CONCAT(CAST(EXTRACT(YEAR from dropoff_datetime) as STRING),"-",CAST(EXTRACT(MONTH from dropoff_datetime) AS STRING)) as dropoff_yearmonth,
      EXTRACT(DATE from dropoff_datetime) as dropoff_date,
      FORMAT_DATE('%A',DATE(dropoff_datetime)) as dropoff_weekday_name,
      EXTRACT(HOUR from dropoff_datetime) as dropoff_hour
      FROM `nyc-tlc.green.trips_2015`
      ) t
      WHERE 
      pickup_datetime BETWEEN '2015-01-01' AND '2016-12-31' 
      AND dropoff_datetime BETWEEN '2015-01-01' AND '2016-12-31'
      AND TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) > 0
      AND passenger_count > 0
      AND trip_distance >= 0 
      AND tip_amount >= 0 
      AND tolls_amount >= 0 
      AND mta_tax >= 0 
      AND fare_amount >= 0
      AND total_amount >= 0
      order by pickup_date DESC
      limit 100000
      """


  result = query_big_query(query_string)
      
  return result.to_dataframe()

clean_df = clean_up_data()

In [9]:
clean_df.columns

Index(['pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag',
       'rate_code', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'trip_distance', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee',
       'total_amount', 'payment_type', 'distance_between_service',
       'time_between_service', 'trip_type', 'time_duration_in_secs',
       'time_duration_in_mins', 'driving_speed_miles_per_hour', 'tip_rate',
       'pickup_year', 'pickup_month', 'pickup_yearmonth', 'pickup_date',
       'pickup_weekday_name', 'pickup_hour', 'dropoff_year', 'dropoff_month',
       'dropoff_yearmonth', 'dropoff_date', 'dropoff_weekday_name',
       'dropoff_hour'],
      dtype='object')

### Discussion of the Clean Data

As we can the see the cleaned data is far more usable.  Typically, I would have performed analysis across a spark cluster or pandas if the resource requirements were not too great. 

Another approach would have been to export the table to another bigtable instance.   I may experiment with this approach if I have time. 


For the sake of the problem given to me, I include the table above as a tmp table within the following working queries.  

In [10]:
def total_passenger_total_ammount():

    query_string = """
        #standardSQL
        select timestamp_trunc(int, month) month, 
        count(pickup_datetime) rides,
        sum(passenger_count) as passenger_count,
        avg(passenger_count) as avg_passenger_count,
        sum(total_amount) as total_amount,
        avg(total_amount) as avg_total_amount
        from `nyc-tlc.green.trips_2015`, 
        unnest(generate_timestamp_array(
        pickup_datetime, 
        dropoff_datetime, 
        interval 1 hour)) int
        where pickup_datetime BETWEEN '2015-02-01' AND '2015-04-30'
        group by month
        order by month
         """

    query_string_2 = """
        with clean as (      
            SELECT 
            t.*,
            FROM
            (
            SELECT *,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) as time_duration_in_secs,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,MINUTE) as time_duration_in_mins,
            ROUND(trip_distance/TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND),2)*3600 as driving_speed_miles_per_hour,
            (CASE WHEN total_amount=0 THEN 0
            ELSE ROUND(tip_amount*100/total_amount,2) END) as tip_rate,
            EXTRACT(YEAR from pickup_datetime) as pickup_year,
            EXTRACT(MONTH from pickup_datetime) as pickup_month,
            CONCAT(CAST(EXTRACT(YEAR from pickup_datetime) as STRING),"-",CAST(EXTRACT(MONTH from pickup_datetime) AS STRING)) as pickup_yearmonth,
            EXTRACT(DATE from pickup_datetime) as pickup_date,
            FORMAT_DATE('%A',DATE(pickup_datetime)) as pickup_weekday_name,
            EXTRACT(HOUR from pickup_datetime) as pickup_hour,
            EXTRACT(YEAR from dropoff_datetime) as dropoff_year,
            EXTRACT(MONTH from dropoff_datetime) as dropoff_month,
            CONCAT(CAST(EXTRACT(YEAR from dropoff_datetime) as STRING),"-",CAST(EXTRACT(MONTH from dropoff_datetime) AS STRING)) as dropoff_yearmonth,
            EXTRACT(DATE from dropoff_datetime) as dropoff_date,
            FORMAT_DATE('%A',DATE(dropoff_datetime)) as dropoff_weekday_name,
            EXTRACT(HOUR from dropoff_datetime) as dropoff_hour
            FROM `nyc-tlc.green.trips_2015`
            /* filter by latitude & longitude that are within the correct range */
            WHERE 
                ((pickup_latitude BETWEEN -90 AND 90) AND
                (pickup_longitude BETWEEN -180 AND 180)) 
            AND
                ((dropoff_latitude BETWEEN -90 AND 90) AND
                (dropoff_longitude BETWEEN -180 AND 180))
            ) t
            /* find the boroughs and zone names for dropoff locations */
            INNER JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` tz_do ON 
            (ST_DWithin(tz_do.zone_geom,ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0))
            /* find the boroughs and zone names for pickup locations */
            INNER JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` tz_pu ON 
            (ST_DWithin(tz_pu.zone_geom,ST_GeogPoint(pickup_longitude, pickup_latitude), 0))
            WHERE 
            pickup_datetime BETWEEN '2015-01-01' AND '2016-12-31' 
            AND dropoff_datetime BETWEEN '2015-01-01' AND '2016-12-31'
            AND TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) > 0
            AND passenger_count > 0
            AND trip_distance >= 0 
            AND tip_amount >= 0 
            AND tolls_amount >= 0 
            AND mta_tax >= 0 
            AND fare_amount >= 0
            AND total_amount >= 0
            )

            #standardSQL
            select date_trunc(int, MONTH) Month_Datetime,
            count(pickup_datetime) rides,
            sum(passenger_count) as passenger_count,
            avg(passenger_count) as avg_passenger_count,
            sum(total_amount) as total_amount,
            avg(total_amount) as avg_total_amount
            from clean,
            unnest(generate_timestamp_array(
            pickup_datetime, 
            dropoff_datetime, 
            interval 1 hour)) as int
            where pickup_datetime BETWEEN '2015-02-01' AND '2015-04-30'
            group by Month_Datetime
            order by Month_Datetime
            
        
         """

## In theory the interval could be as small as 1 minute.  Doing so could in theory be more accurate, however, it may also overcount the total ammount billed and the passenger count in the group by.

    result = query_big_query(query_string=query_string_2)
    df = result.to_dataframe()
    return df

df = total_passenger_total_ammount()
    

In [11]:
df

Month_Datetime    rides  passenger_count  avg_passenger_count  \
0 2015-02-01 00:00:00+00:00  1600035          2209835             1.381117   
1 2015-03-01 00:00:00+00:00  1763018          2430562             1.378637   
2 2015-04-01 00:00:00+00:00  1652599          2279631             1.379422   

   total_amount  avg_total_amount  
0   23452447.87         14.657459  
1   26096490.67         14.802169  
2   24885815.08         15.058593

### Discussion of the Solution to Problem 1.1

The resulting data can is accurate to the month.  It is also account for cases when rides overlap across datetimes.  For example, if a ride begins at 12:00 but ends at 14:00, the query above will count the ridership at hours 12 and 13.   

The one drawback to this approach is that it can in theory inflate the total_amount value.  If considering 1 minute intervals as opposed to an hour the rate of hour expands the total_amount value too greatly.  The solution to the problem would be to work with the rate to recalculate the total amount per hour, minute, etc.  Thankfully, as most rides are less than an hour long, it is uncecessary for the question posed.  

## Question 1.2

What has been the average hourly passenger count throughout the year?

In [12]:
def hourly_count_through_year():


    query_string_2 = """
        with clean as (      
            SELECT 
            t.*,
            FROM
            (
            SELECT *,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) as time_duration_in_secs,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,MINUTE) as time_duration_in_mins,
            ROUND(trip_distance/TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND),2)*3600 as driving_speed_miles_per_hour,
            (CASE WHEN total_amount=0 THEN 0
            ELSE ROUND(tip_amount*100/total_amount,2) END) as tip_rate,
            EXTRACT(YEAR from pickup_datetime) as pickup_year,
            EXTRACT(MONTH from pickup_datetime) as pickup_month,
            CONCAT(CAST(EXTRACT(YEAR from pickup_datetime) as STRING),"-",CAST(EXTRACT(MONTH from pickup_datetime) AS STRING)) as pickup_yearmonth,
            EXTRACT(DATE from pickup_datetime) as pickup_date,
            FORMAT_DATE('%A',DATE(pickup_datetime)) as pickup_weekday_name,
            EXTRACT(HOUR from pickup_datetime) as pickup_hour,
            EXTRACT(YEAR from dropoff_datetime) as dropoff_year,
            EXTRACT(MONTH from dropoff_datetime) as dropoff_month,
            CONCAT(CAST(EXTRACT(YEAR from dropoff_datetime) as STRING),"-",CAST(EXTRACT(MONTH from dropoff_datetime) AS STRING)) as dropoff_yearmonth,
            EXTRACT(DATE from dropoff_datetime) as dropoff_date,
            FORMAT_DATE('%A',DATE(dropoff_datetime)) as dropoff_weekday_name,
            EXTRACT(HOUR from dropoff_datetime) as dropoff_hour
            FROM `nyc-tlc.green.trips_2015`
            /* filter by latitude & longitude that are within the correct range */
            WHERE 
                ((pickup_latitude BETWEEN -90 AND 90) AND
                (pickup_longitude BETWEEN -180 AND 180)) 
            AND
                ((dropoff_latitude BETWEEN -90 AND 90) AND
                (dropoff_longitude BETWEEN -180 AND 180))
            ) t
            WHERE 
            pickup_datetime BETWEEN '2015-01-01' AND '2016-12-31' 
            AND dropoff_datetime BETWEEN '2015-01-01' AND '2016-12-31'
            AND passenger_count > 0
            AND trip_distance >= 0 
            AND tip_amount >= 0 
            AND tolls_amount >= 0 
            AND mta_tax >= 0 
            AND fare_amount >= 0
            AND total_amount >= 0
            )
            #standardSQL
            select date_trunc(int, hour) Hour_Datetime,
            count(pickup_datetime) rides,
            sum(passenger_count) as passenger_count,
            avg(passenger_count) as avg_passenger_count,
            from clean,
            unnest(generate_timestamp_array(
            pickup_datetime, 
            dropoff_datetime, 
            interval 1 hour)) as int
            group by Hour_Datetime
            order by Hour_Datetime
            
            """

## In theory the interval could be as small as 1 minute.  Doing so could in theory be more accurate, however, it may also overcount the total ammount billed and the passenger count in the group by.

    result = query_big_query(query_string=query_string_2)
    df = result.to_dataframe()
    return df

df = hourly_count_through_year()
    

In [13]:
df

Hour_Datetime  rides  passenger_count  avg_passenger_count
0    2015-01-01 00:00:00+00:00   6458             9605             1.487303
1    2015-01-01 01:00:00+00:00   6871            10175             1.480862
2    2015-01-01 02:00:00+00:00   6487             9851             1.518576
3    2015-01-01 03:00:00+00:00   5913             8822             1.491967
4    2015-01-01 04:00:00+00:00   5095             7663             1.504024
...                        ...    ...              ...                  ...
4362 2015-07-01 18:00:00+00:00      7                8             1.142857
4363 2015-07-01 19:00:00+00:00      5                6             1.200000
4364 2015-07-01 20:00:00+00:00      5                6             1.200000
4365 2015-07-01 21:00:00+00:00      4                5             1.250000
4366 2015-07-01 22:00:00+00:00      1                1             1.000000

[4367 rows x 4 columns]

### Discussion of Question 1.2

At first, I was confused why the data terminated at 2015-07-01.  I thought, I had made a mistake. I tested the data on the yellow line data without error.  I then reviewed the big query table to find that it actually terminates at 2015-07-01. 



## Question 1.3

* What has been the change/delta in total amount billed over days? What we would like see is how much (positive or negative)?

In [14]:
def difference_by_day():


    query_string_2 = """
        with clean as (      
            SELECT 
            t.*,
            FROM
            (
            SELECT *,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) as time_duration_in_secs,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,MINUTE) as time_duration_in_mins,
            ROUND(trip_distance/TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND),2)*3600 as driving_speed_miles_per_hour,
            (CASE WHEN total_amount=0 THEN 0
            ELSE ROUND(tip_amount*100/total_amount,2) END) as tip_rate,
            EXTRACT(YEAR from pickup_datetime) as pickup_year,
            EXTRACT(MONTH from pickup_datetime) as pickup_month,
            CONCAT(CAST(EXTRACT(YEAR from pickup_datetime) as STRING),"-",CAST(EXTRACT(MONTH from pickup_datetime) AS STRING)) as pickup_yearmonth,
            EXTRACT(DATE from pickup_datetime) as pickup_date,
            FORMAT_DATE('%A',DATE(pickup_datetime)) as pickup_weekday_name,
            EXTRACT(HOUR from pickup_datetime) as pickup_hour,
            EXTRACT(YEAR from dropoff_datetime) as dropoff_year,
            EXTRACT(MONTH from dropoff_datetime) as dropoff_month,
            CONCAT(CAST(EXTRACT(YEAR from dropoff_datetime) as STRING),"-",CAST(EXTRACT(MONTH from dropoff_datetime) AS STRING)) as dropoff_yearmonth,
            EXTRACT(DATE from dropoff_datetime) as dropoff_date,
            FORMAT_DATE('%A',DATE(dropoff_datetime)) as dropoff_weekday_name,
            EXTRACT(HOUR from dropoff_datetime) as dropoff_hour
            FROM `nyc-tlc.green.trips_2015`
            /* filter by latitude & longitude that are within the correct range */
            WHERE 
                ((pickup_latitude BETWEEN -90 AND 90) AND
                (pickup_longitude BETWEEN -180 AND 180)) 
            AND
                ((dropoff_latitude BETWEEN -90 AND 90) AND
                (dropoff_longitude BETWEEN -180 AND 180))
            ) t
            /* find the boroughs and zone names for dropoff locations */
            INNER JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` tz_do ON 
            (ST_DWithin(tz_do.zone_geom,ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0))
            /* find the boroughs and zone names for pickup locations */
            INNER JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` tz_pu ON 
            (ST_DWithin(tz_pu.zone_geom,ST_GeogPoint(pickup_longitude, pickup_latitude), 0))
            WHERE 
            pickup_datetime BETWEEN '2015-01-01' AND '2016-12-31' 
            AND dropoff_datetime BETWEEN '2015-01-01' AND '2016-12-31'
            AND TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) > 0
            AND passenger_count > 0
            AND trip_distance >= 0 
            AND tip_amount >= 0 
            AND tolls_amount >= 0 
            AND mta_tax >= 0 
            AND fare_amount >= 0
            AND total_amount >= 0
            ),
            daily as(
            #standardSQL
            select date_trunc(int, DAY) DAY_Datetime,
            count(pickup_datetime) rides,
            sum(total_amount) as total_amount,
            avg(total_amount) as avg_total_amount
            from clean,
            unnest(generate_timestamp_array(
            pickup_datetime, 
            dropoff_datetime, 
            interval 1 hour)) as int
            group by DAY_Datetime
            order by DAY_Datetime
            )

            select *,
            total_amount - LAG(total_amount) OVER (ORDER BY DAY_Datetime) AS Difference
            FROM daily;
            
        
         """

## In theory the interval could be as small as 1 minute.  Doing so could in theory be more accurate, however, it may also overcount the total ammount billed and the passenger count in the group by.

    result = query_big_query(query_string=query_string_2)
    df = result.to_dataframe()
    return df

df = difference_by_day()
    

In [15]:
df

DAY_Datetime  rides  total_amount  avg_total_amount  \
0   2015-03-11 00:00:00+00:00  51349     785893.71         15.304947   
1   2015-04-30 00:00:00+00:00  54682     841361.04         15.386435   
2   2015-01-02 00:00:00+00:00  44343     613961.10         13.845728   
3   2015-02-17 00:00:00+00:00  42433     607052.73         14.306147   
4   2015-04-01 00:00:00+00:00  53156     779533.09         14.665007   
..                        ...    ...           ...               ...   
177 2015-01-15 00:00:00+00:00  48662     703349.06         14.453764   
178 2015-02-24 00:00:00+00:00  51009     735429.01         14.417632   
179 2015-03-07 00:00:00+00:00  74557    1113229.81         14.931258   
180 2015-03-10 00:00:00+00:00  52225     798785.92         15.295087   
181 2015-05-29 00:00:00+00:00  62678     980866.89         15.649301   

     Difference  
0     -12892.21  
1      92337.70  
2    -349817.92  
3     -61255.96  
4      22250.61  
..          ...  
177    71308.53  
178   -63239.31  
179   120360.35  
180   107550.57  
181   150118.05  

[182 rows x 5 columns]

### Discussion of Question 1.3

The approach is almost identical to my previous answers.  The only major difference is the inclusion of the lag function to determine the running differences.  

## Question 1.4

* What hour of the day has seen the longest rides in April?


### Date_trunc Method

Date_trunc will return the requested results per date per hour.

In [16]:
def longest_rides_per_hour():


    query_string_2 = """
        with clean as (      
            SELECT 
            t.*,
            FROM
            (
            SELECT *,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) as time_duration_in_secs,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,MINUTE) as time_duration_in_mins,
            ROUND(trip_distance/TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND),2)*3600 as driving_speed_miles_per_hour,
            (CASE WHEN total_amount=0 THEN 0
            ELSE ROUND(tip_amount*100/total_amount,2) END) as tip_rate,
            EXTRACT(YEAR from pickup_datetime) as pickup_year,
            EXTRACT(MONTH from pickup_datetime) as pickup_month,
            CONCAT(CAST(EXTRACT(YEAR from pickup_datetime) as STRING),"-",CAST(EXTRACT(MONTH from pickup_datetime) AS STRING)) as pickup_yearmonth,
            EXTRACT(DATE from pickup_datetime) as pickup_date,
            FORMAT_DATE('%A',DATE(pickup_datetime)) as pickup_weekday_name,
            EXTRACT(HOUR from pickup_datetime) as pickup_hour,
            EXTRACT(YEAR from dropoff_datetime) as dropoff_year,
            EXTRACT(MONTH from dropoff_datetime) as dropoff_month,
            CONCAT(CAST(EXTRACT(YEAR from dropoff_datetime) as STRING),"-",CAST(EXTRACT(MONTH from dropoff_datetime) AS STRING)) as dropoff_yearmonth,
            EXTRACT(DATE from dropoff_datetime) as dropoff_date,
            FORMAT_DATE('%A',DATE(dropoff_datetime)) as dropoff_weekday_name,
            EXTRACT(HOUR from dropoff_datetime) as dropoff_hour
            FROM `nyc-tlc.green.trips_2015`
            /* filter by latitude & longitude that are within the correct range */
            WHERE 
                ((pickup_latitude BETWEEN -90 AND 90) AND
                (pickup_longitude BETWEEN -180 AND 180)) 
            AND
                ((dropoff_latitude BETWEEN -90 AND 90) AND
                (dropoff_longitude BETWEEN -180 AND 180))
            ) t
            /* find the boroughs and zone names for dropoff locations */
            INNER JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` tz_do ON 
            (ST_DWithin(tz_do.zone_geom,ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0))
            /* find the boroughs and zone names for pickup locations */
            INNER JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` tz_pu ON 
            (ST_DWithin(tz_pu.zone_geom,ST_GeogPoint(pickup_longitude, pickup_latitude), 0))
            WHERE 
            pickup_datetime BETWEEN '2015-01-01' AND '2016-12-31' 
            AND dropoff_datetime BETWEEN '2015-01-01' AND '2016-12-31'
            AND TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) > 0
            AND passenger_count > 0
            AND trip_distance >= 0 
            AND tip_amount >= 0 
            AND tolls_amount >= 0 
            AND mta_tax >= 0 
            AND fare_amount >= 0
            AND total_amount >= 0
            ),
            hourly as(
            #standardSQL
            select date_trunc(int, hour) DAY_Datetime,
            count(pickup_datetime) rides,
            sum(total_amount) as total_amount,
            avg(total_amount) as avg_total_amount,
            avg(time_duration_in_mins) as avg_trip_duration_mins,
            max(time_duration_in_mins) as max_time_duration_mins,
            avg(trip_distance) as avg_trip_distance,
            max(trip_distance) as max_trip_distance
            from clean,
            unnest(generate_timestamp_array(
            pickup_datetime, 
            dropoff_datetime, 
            interval 1 hour)) as int
            where pickup_datetime BETWEEN '2015-04-01' AND '2015-04-30' 
            AND dropoff_datetime BETWEEN '2015-04-01' AND '2015-04-30' 
            group by DAY_Datetime
            order by DAY_Datetime
            )

            select *,
            FROM hourly
            order by avg_trip_distance DESC, avg_trip_duration_mins DESC;
            
        
         """

## In theory the interval could be as small as 1 minute.  Doing so could in theory be more accurate, however, it may also overcount the total ammount billed and the passenger count in the group by.

    result = query_big_query(query_string=query_string_2)
    df = result.to_dataframe()
    return df

df = longest_rides_per_hour()
    

In [17]:
df

DAY_Datetime  rides  total_amount  avg_total_amount  \
0   2015-04-22 05:00:00+00:00    437       8832.02         20.210572   
1   2015-04-21 05:00:00+00:00    408       7779.64         19.067745   
2   2015-04-08 05:00:00+00:00    417       7911.22         18.971751   
3   2015-04-24 05:00:00+00:00    572      11226.31         19.626416   
4   2015-04-01 05:00:00+00:00    371       6904.21         18.609730   
..                        ...    ...           ...               ...   
691 2015-04-07 20:00:00+00:00   3355      43535.74         12.976376   
692 2015-04-29 20:00:00+00:00   3599      47430.23         13.178725   
693 2015-04-22 18:00:00+00:00   4136      59788.97         14.455747   
694 2015-04-07 18:00:00+00:00   3621      50035.31         13.818092   
695 2015-04-07 19:00:00+00:00   3954      54699.25         13.833902   

     avg_trip_duration_mins  max_time_duration_mins  avg_trip_distance  \
0                168.228833                    1434           4.680458   
1                157.294118                    1438           4.563897   
2                178.841727                    1439           4.525180   
3                187.965035                    1439           4.501678   
4                 19.574124                    1402           4.483531   
..                      ...                     ...                ...   
691               32.518331                    1439           2.451830   
692               16.185051                    1438           2.426913   
693               29.315039                    1437           2.410539   
694               32.520851                    1439           2.334471   
695               29.794133                    1439           2.280878   

     max_trip_distance  
0                38.68  
1                22.88  
2                26.10  
3                23.00  
4                21.81  
..                 ...  
691              42.14  
692              44.00  
693              60.83  
694              54.83  
695              54.83  

[696 rows x 8 columns]

#### Discussion of Date Trunc Method

I wanted to see if avg_trip_duration would correlate with avg_trip_distance.  

In this sample it does seem to. I would like to test the distributions later for correlation. 

To answer the question, 2014-04-22 at 5:00 am recorded the longest trips.  Interestingly, most of longest trips are at 5:00.  I expected the evening rush hour to record a larger number of results. As this is the green line, it would make sense that a large majority of taking the vehicle to the airport or to a determined destination as opposed to randomly hailing a yellow cab.  

Review the code below for a results below for a more succint table.  


### The Extract Method

Extract will record the values by hour of the 24 hour clock.  It will aggregate accordinly.

In [18]:
def longest_rides_per_hour_etracted():


    query_string_2 = """
        with clean as (      
            SELECT 
            t.*,
            FROM
            (
            SELECT *,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) as time_duration_in_secs,
            TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,MINUTE) as time_duration_in_mins,
            ROUND(trip_distance/TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND),2)*3600 as driving_speed_miles_per_hour,
            (CASE WHEN total_amount=0 THEN 0
            ELSE ROUND(tip_amount*100/total_amount,2) END) as tip_rate,
            EXTRACT(YEAR from pickup_datetime) as pickup_year,
            EXTRACT(MONTH from pickup_datetime) as pickup_month,
            CONCAT(CAST(EXTRACT(YEAR from pickup_datetime) as STRING),"-",CAST(EXTRACT(MONTH from pickup_datetime) AS STRING)) as pickup_yearmonth,
            EXTRACT(DATE from pickup_datetime) as pickup_date,
            FORMAT_DATE('%A',DATE(pickup_datetime)) as pickup_weekday_name,
            EXTRACT(HOUR from pickup_datetime) as pickup_hour,
            EXTRACT(YEAR from dropoff_datetime) as dropoff_year,
            EXTRACT(MONTH from dropoff_datetime) as dropoff_month,
            CONCAT(CAST(EXTRACT(YEAR from dropoff_datetime) as STRING),"-",CAST(EXTRACT(MONTH from dropoff_datetime) AS STRING)) as dropoff_yearmonth,
            EXTRACT(DATE from dropoff_datetime) as dropoff_date,
            FORMAT_DATE('%A',DATE(dropoff_datetime)) as dropoff_weekday_name,
            EXTRACT(HOUR from dropoff_datetime) as dropoff_hour
            FROM `nyc-tlc.green.trips_2015`
            /* filter by latitude & longitude that are within the correct range */
            WHERE 
                ((pickup_latitude BETWEEN -90 AND 90) AND
                (pickup_longitude BETWEEN -180 AND 180)) 
            AND
                ((dropoff_latitude BETWEEN -90 AND 90) AND
                (dropoff_longitude BETWEEN -180 AND 180))
            ) t
            /* find the boroughs and zone names for dropoff locations */
            INNER JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` tz_do ON 
            (ST_DWithin(tz_do.zone_geom,ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0))
            /* find the boroughs and zone names for pickup locations */
            INNER JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` tz_pu ON 
            (ST_DWithin(tz_pu.zone_geom,ST_GeogPoint(pickup_longitude, pickup_latitude), 0))
            WHERE 
            pickup_datetime BETWEEN '2015-01-01' AND '2016-12-31' 
            AND dropoff_datetime BETWEEN '2015-01-01' AND '2016-12-31'
            AND TIMESTAMP_DIFF(dropoff_datetime,pickup_datetime,SECOND) > 0
            AND passenger_count > 0
            AND trip_distance >= 0 
            AND tip_amount >= 0 
            AND tolls_amount >= 0 
            AND mta_tax >= 0 
            AND fare_amount >= 0
            AND total_amount >= 0
            ),
            hourly as(
            #standardSQL
            select EXTRACT(HOUR from int) DAY_Datetime,
            count(pickup_datetime) rides,
            sum(total_amount) as total_amount,
            avg(total_amount) as avg_total_amount,
            avg(time_duration_in_mins) as avg_trip_duration_mins,
            max(time_duration_in_mins) as max_time_duration_mins,
            avg(trip_distance) as avg_trip_distance,
            max(trip_distance) as max_trip_distance
            from clean,
            unnest(generate_timestamp_array(
            pickup_datetime, 
            dropoff_datetime, 
            interval 1 hour)) as int
            where pickup_datetime BETWEEN '2015-04-01' AND '2015-04-30' 
            AND dropoff_datetime BETWEEN '2015-04-01' AND '2015-04-30' 
            group by DAY_Datetime
            order by DAY_Datetime
            )

            select *,
            FROM hourly
            order by avg_trip_distance DESC, avg_trip_duration_mins DESC;
            
        
         """

## In theory the interval could be as small as 1 minute.  Doing so could in theory be more accurate, however, it may also overcount the total ammount billed and the passenger count in the group by.

    result = query_big_query(query_string=query_string_2)
    df = result.to_dataframe()
    return df

df = longest_rides_per_hour_etracted()
    

In [19]:
df

DAY_Datetime   rides  total_amount  avg_total_amount  \
0              6   23407     406676.31         17.374132   
1              5   18530     323255.01         17.444955   
2              4   28836     446638.44         15.488918   
3              7   43269     679401.77         15.701814   
4              3   35060     515585.82         14.705813   
5             23   89991    1347089.17         14.969154   
6              9   67749    1045347.79         15.429715   
7              0   75056    1107289.88         14.752850   
8              2   45003     645698.17         14.347892   
9              8   63888     987487.01         15.456533   
10             1   59383     859156.03         14.468047   
11            22   97416    1444937.04         14.832646   
12            10   62846     941869.28         14.986941   
13            11   61425     922729.92         15.022058   
14            13   62655     948215.73         15.133920   
15            12   62336     940571.12         15.088731   
16            14   71541    1087273.14         15.197902   
17            15   80211    1228121.11         15.311131   
18            16   86157    1371038.06         15.913252   
19            21   99867    1441899.37         14.438197   
20            17   96276    1501988.38         15.600860   
21            20  104678    1482750.27         14.164870   
22            19  106840    1557368.06         14.576639   
23            18  107685    1609981.95         14.950847   

    avg_trip_duration_mins  max_time_duration_mins  avg_trip_distance  \
0               120.617807                    1439           4.013681   
1               147.563195                    1439           3.994335   
2                98.029338                    1439           3.406666   
3                72.091081                    1439           3.254081   
4                81.675670                    1439           3.136565   
5                42.006667                    1439           3.023357   
6                51.702977                    1439           3.018475   
7                44.033082                    1439           3.014348   
8                65.855010                    1439           2.993119   
9                54.133186                    1439           2.992707   
10               52.410690                    1439           2.970133   
11               39.692453                    1439           2.969648   
12               54.270678                    1439           2.966649   
13               55.576703                    1439           2.955951   
14               55.472125                    1439           2.954582   
15               55.376476                    1439           2.944434   
16               51.073734                    1439           2.923313   
17               47.997295                    1439           2.896561   
18               46.012396                    1439           2.860492   
19               38.715141                    1439           2.824972   
20               42.422130                    1439           2.747579   
21               37.491994                    1439           2.710590   
22               37.460680                    1439           2.670077   
23               38.261262                    1439           2.662435   

    max_trip_distance  
0               58.49  
1               87.90  
2               87.90  
3               70.94  
4               87.91  
5              297.06  
6               70.94  
7              134.26  
8               87.91  
9               70.94  
10             134.26  
11             297.06  
12             375.64  
13             375.64  
14             375.64  
15             375.64  
16             375.64  
17             375.64  
18             375.64  
19             297.06  
20             375.64  
21             297.06  
22             297.06  
23             375.64

#### Discussion of the Extract Method

The longest time duration is recorded at 5 am.  The distance between 5 and 6 am are marginal.  